# Central Puget Sound Data Trends (2010 - 2023)

Tiernan Martin

In [ ]:

library(tidyverse)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'scales'

The following object is masked from 'package:purrr':

    discard

The following object is masked from 'package:readr':

    col_factor


Attaching package: 'janitor'

The following objects are masked from 'package:stats':

    chisq.test, fisher.test

here() starts at C:/Users/tiern/Documents/R/2024-transit-oriented-development-bill

## Data

This analysis uses data from Washington State’s Office of Financial Management: [Historical estimates of April 1 population and housing for the state, counties, and cities](https://ofm.wa.gov/washington-data-research/population-demographics/population-estimates/historical-estimates-april-1-population-and-housing-state-counties-and-cities)

### Change in Population

In [ ]:

pop_fp <- here("data/ofm_april1_postcensal_estimates_pop_1960-present.xlsx")

pop_df <- read_excel(pop_fp, sheet = "Population", skip = 3) |> 
  clean_names()


New names:
• `` -> `...16`
• `` -> `...18`
• `` -> `...20`
• `` -> `...22`
• `` -> `...24`
• `` -> `...26`
• `` -> `...28`
• `` -> `...30`
• `` -> `...32`
• `` -> `...34`
• `` -> `...36`
• `` -> `...38`
• `` -> `...40`
• `` -> `...42`
• `` -> `...44`
• `` -> `...46`
• `` -> `...48`
• `` -> `...50`
• `` -> `...52`
• `` -> `...54`
• `` -> `...56`
• `` -> `...58`
• `` -> `...60`
• `` -> `...62`
• `` -> `...64`
• `` -> `...66`
• `` -> `...68`
• `` -> `...70`
• `` -> `...72`
• `` -> `...74`
• `` -> `...76`
• `` -> `...78`
• `` -> `...80`
• `` -> `...82`
• `` -> `...84`
• `` -> `...86`
• `` -> `...88`
• `` -> `...90`
• `` -> `...92`
• `` -> `...94`
• `` -> `...96`
• `` -> `...98`
• `` -> `...100`
• `` -> `...102`
• `` -> `...104`
• `` -> `...106`
• `` -> `...108`
• `` -> `...110`
• `` -> `...112`
• `` -> `...114`
• `` -> `...116`
• `` -> `...118`
• `` -> `...120`
• `` -> `...122`
• `` -> `...124`
• `` -> `...126`
• `` -> `...128`

ℹ In argument: `value = as.numeric(value)`.
Caused by warning:
! NAs introduced by coercion

# A tibble: 2 × 5
  pugetsound pop_2010 pop_2023 pop_change pop_change_pct
  <lgl>         <dbl>    <dbl>      <dbl>          <dbl>
1 FALSE        470300   554600      84300           0.18
2 TRUE        1933400  2347800     414400           0.21

In [ ]:

pugetsound_pop_growth_2010_2023 <- pop_growth |> filter(pugetsound) |> pluck("pop_change")

saveRDS(pugetsound_pop_growth_2010_2023,
        here("data_outputs/pugetsound_pop_growth_2010_2023.rds"))

outside_pop_growth_2010_2023 <- pop_growth |> filter(!pugetsound) |> pluck("pop_change")

saveRDS(outside_pop_growth_2010_2023,
        here("data_outputs/outside_pop_growth_2010_2023.rds"))

pop_change_pct <- pop_growth |> filter(pugetsound) |> pluck("pop_change_pct")

pop_change_text <- glue("{comma(pugetsound_pop_growth_2010_2023)} people (a {percent(pop_change_pct)} increase)")


### Change in Housing Units

In [ ]:

uh_fp <- here("data/ofm_april1_postcensal_estimates_housing_1980_1990-present.xlsx")

hu_df <- read_excel(uh_fp, sheet = "Housing Units", skip = 3) |>  
  clean_names()

hu_growth_2010_2023 <- hu_df |> 
  filter(filter %in% "1") |> 
  pivot_longer(cols = x1980_census_count_of_total_housing_units:last_col()) |> 
  mutate(value = as.numeric(value),
         pugetsound = case_when(
           county %in% c("King",
                         "Snohomish",
                         "Kitsap",
                         "Pierce") ~ TRUE,
           TRUE ~ FALSE
         )) |> 
  # filter(str_detect(name,"postcensal")) |> 
  filter(str_detect(name,"total")) |> 
  filter(str_detect(name, "2010|2023")) |>  
  mutate(name = str_c("hu_", str_extract(name,"\\d{4}"))) |> 
  pivot_wider(names_from = name,
              values_from = value) |> 
  group_by(pugetsound) |> 
  summarise(hu = sum(hu_2023) - sum(hu_2010))

hu_growth_2010_2023


# A tibble: 2 × 2
  pugetsound     hu
  <lgl>       <dbl>
1 FALSE      179786
2 TRUE       276177

In [ ]:

pugetsound_hu_growth_2010_2023 <- hu_growth_2010_2023 |> filter(pugetsound) |> pluck("hu")

saveRDS(pugetsound_hu_growth_2010_2023,
        here("data_outputs/pugetsound_hu_growth_2010_2023.rds"))


outside_hu_pop_growth_2010_2023 <- hu_growth_2010_2023 |> filter(!pugetsound) |> pluck("hu")

saveRDS(outside_hu_pop_growth_2010_2023,
        here("data_outputs/outside_hu_growth_2010_2023.rds"))
